# Correction of Bimolecular Recombination-Limited Charge Extraction Experiments in Low-Mobility Diodes

## Supporting Code

_Austin M. Kay$^1$, Oskar J. Sandberg$^2$, Drew B. Riley$^1$, Paul Meredith$^1$, & Ardalan Armin$^1$_

$^1$Sustainable Advanced Materials (Sêr SAM), Centre for Integrative Semiconductor Materials (CISM), Swansea University Bay Campus, Crymlyn Burrows, SA1 8EN, United Kingdom

$^2$ Physics, Faculty of Science and Engineering, Åbo Akademi University, 20500 Turku, Finland


Email: (AMK) 954708@swansea.ac.uk; (OJS) oskar.sandberg@abo.fi; (AA) ardalan.armin@swansea.ac.uk

***

<a id = 'ToC' ></a>
## Table of Contents

1.&emsp;&emsp;[Prerequisites](#Prelims)

2.&emsp;&emsp;[Effective Built-In Voltage Determination](#Vbi)

3.&emsp;&emsp;[Bimolecular Recombination Rate Constant Determination](#BetaDetermination)

&emsp;&emsp; 3.1. &emsp;&emsp;[Code for Actually Determining the Rate](#BetaCode)

&emsp;&emsp; 3.2. &emsp;&emsp;[User Interface Compiling](#BetaUIComp)

&emsp;&emsp; 3.3. &emsp;&emsp;[The User Interface](#BetaUI)

<a id = 'Prelims' ></a>
## 1. Prerequisites
[Return to Table of Contents](#ToC)
<br/><br/> 
To begin, the libraries and tools needed by the script are imported:
<br/><br/> 

In [1]:
from os import path, getcwd, listdir
from numpy import *
from ipywidgets import *
from scipy.constants import epsilon_0, k, e
import matplotlib.pyplot as plt
from pandas import read_excel, DataFrame
import random as rdm
from decimal import Decimal

epsilon_0 = epsilon_0 / 100   # convert to F/cm

<br/><br/> 
The current working directory, and data folders are identified using:
<br/><br/> 

In [2]:
Current_Working_Directory = getcwd()

Beta_Data_Folder = path.join( Current_Working_Directory, 'Beta_vs_n_data' )

Beta_Files = [ File for File in listdir( Beta_Data_Folder ) if '.xlsx' in File ] # Assume data is in Excel files

<br/><br/> 
Determine the filepaths to each of these files:
<br/><br/> 

In [3]:
Beta_Filepaths = { File : path.join( Beta_Data_Folder, File ) for File in Beta_Files }

<a id = 'Vbi' ></a>
## 2. Effective Built-In Voltage Determination

[Return to Table of Contents](#ToC)

Insert values for parameters:
<br/><br/> 

In [4]:
E_g = 1.2

V_bi0 = E_g      # V, assuming Ohmic contacts with perfectly selective injection barriers - no losses

eps_r = 3

p_an = 1e20      # cm-3

n_cat = 1e20     # cm-3

d = 100e-7       # cm

T = 293.15

kTq = k * T / e  # V

<br/><br/> 
Specify the iterative control parameters:
<br/><br/> 

In [5]:
Convergence_Criterion = 1e-6

V_bi = V_bi0     # V, zeroth-order approximation, assumes that 0<V_bi<V_bi_0 

delta_V_bi = V_bi0

<br/><br/> 
The following helper function is then defined for determining the next iteration of the built-in votlage:
<br/><br/> 

In [6]:
def V_bi_next( V_bi_in, V_bi0, kTq, p_an, n_cat, eps_r, d ):
    
    Middle_Term = - 2 * kTq * log( e * d ** 2 * ( p_an * n_cat ) ** 0.5 / 2 / eps_r / epsilon_0 / kTq )
    
    Final_Term = + 4 * kTq * log( V_bi_in / kTq )
    
    return V_bi0 + Middle_Term + Final_Term

<br/><br/> 
The following code will then iterate until converged, giving a value for the effective built-in votlage:
<br/><br/> 

In [7]:
while delta_V_bi > Convergence_Criterion:
    
    New_V_bi = V_bi_next( V_bi, V_bi0, kTq, p_an, n_cat, eps_r, d )
            
    delta_V_bi = abs( New_V_bi - V_bi ) / V_bi
    
    V_bi = New_V_bi

<br/><br/> 
The effective built-in votlage is then given by:
<br/><br/> 

In [8]:
print( V_bi )

0.9789204830584324


<a id = 'BetaDetermination' ></a>
## 3. Bimolecular Recombination Rate Constant Determination

[Return to Table of Contents](#ToC)
<br/><br/> 
In this section, the code to determine the actual bimolecular recombination rate, the code to create and compile a user interface, and the user interface itself are presented.
<a id = 'BetaCode' ></a>
### 3.1. Code for Actually Determining the Rate
[Return to Table of Contents](#ToC)
<br/><br/> 
To begin, the code for calculating the corrected rate of bimolecular recombination using the analytical model presented in our work is presented. This includes the calculation of the exptected extracted charge density using:
<br/><br/> 

In [9]:
def n_CE( t_tr, G_light, beta ):
    
    """Calculate the extracted charge, n_CE, in the analytical model described in our work, using the bimolecular 
    
    recombination rate constant beta, the light intenisty G_light, and the transit time, t_tr."""
    
    n_oc = sqrt( G_light / beta )
    
    prefactor = 2 / beta / t_tr
    
    A = beta * t_tr * n_oc
    
    return prefactor * ( ( 1 + 1 / A ) * log( 1 + A ) - 1 ) 

<br/><br/> 
Next, the correction factor that may be applied to the obtained bimolecular recombination rate constant to give a more accurate value (particularly at higher light intensities) may be calculated using:
<br/><br/> 

In [10]:
def beta_correction( n_CE, t_tr, G_light, beta ):
    
    """The correction factor, f, which can be multiplied with the observed bimolecular recombination rate constant beta (at
    
    a particular set of G_light and n_CE values) to give the corrected beta_value. Do this using the transit time, t_tr."""
    
    n_oc = sqrt( G_light / beta )
    
    prefactor = 2 * n_CE / G_light / t_tr
    
    A = beta * t_tr * n_oc
    
    return prefactor * ( ( 1 + 1 / A ) * log( 1 + A ) - 1 ) 

<br/><br/> 
On the other hand, the following function may be used to evaluate what bimolecular recombination rate constant might be observed under an actual $\beta$ value:
<br/><br/> 

In [11]:
def beta_measured( n_CE, G_light, beta, t_tr ):
    
    """Calculate what apparent bimolecular recombination rate would be measured using the actual bimolecular recombination 
    
    rate, the extracted carrier density, the generation rate, and the transit time."""
    
    n_oc = sqrt( G_light / beta )
    
    A = beta * t_tr * n_oc
    
    denominator = 2 * n_CE * ( ( 1 + 1 / A ) * log( 1 + A ) - 1 )
    
    denominator = 2 * n_CE * ( ( 1 + 1 / ( t_tr * sqrt( G_light * beta ) ) ) * log( 1 + t_tr * sqrt( G_light * beta ) ) - 1 )

    return beta * t_tr * G_light / denominator

<br/><br/> 
Using the above function, the below function simulates expected $\beta$ curves for 'N_samples' randomly-selected pairings of input $\beta$ and $t_{\mathrm{tr}}$ before determining the residual sum of the squares - this is used by the next function to "fit" the three data sets and estimate the actual values of the parameters:
<br/><br/> 

In [12]:
def beta_measured_fitter( n_CEs, G_lights, measured_betas, N_samples, lower_beta, upper_beta, lower_t_tr, upper_t_tr ):
    
    """Assume that upper and lower beta values are logarithmically scaled."""
    
    N_samples = int( N_samples )
    
    sample_betas = logspace( log10( lower_beta ), log10( upper_beta ), N_samples )
    
    sample_t_trs = logspace( log10( lower_t_tr ), log10( upper_t_tr ), N_samples )

    Treated_Pairs = {}
    
    for j in range( N_samples ):
        
        Unique_Pair = False
        
        # First Step: Ensure we have a unique pair 
        
        while Unique_Pair == False:
        
            beta_j = random.choice( sample_betas )
            
            t_tr_j = random.choice( sample_t_trs )
            
            Pair = str( beta_j) + ', ' + str( t_tr_j )
            
            if Pair in Treated_Pairs.keys():
                
                Unique_Pair = False
                
            else:
                
                Treated_Pairs[ Pair ] = [ beta_j, t_tr_j ]
                
                Unique_Pair = True
                
        # Next Step: Simulated the measured betas for this pairing: 
                
        Simulated_betas = beta_measured( n_CEs, G_lights, beta_j, t_tr_j )
        
        # Calculate the Residual Sum of the Squares
        
        RSS = sum( [ abs( Simulated_betas[ k ] - measured_betas[ k ] ) ** 2 for k in range( len( n_CEs ) ) ] )
        
        Treated_Pairs[ Pair ].append( RSS )
        
    return Treated_Pairs

<br/><br/> 
For a specified number of repeats (which "zoom in" on the input parameter values that give the minimal residual sum of squares), use the above function estimate $\beta$ and $t_{\mathrm{tr}}$ for a given spectrum:
<br/><br/> 

In [13]:
def Repeated_Beta_fitter( n_CEs, 
                         
                         G_lights, 
                         
                         measured_betas, 
                         
                         N_samples, 
                         
                         N_repeats,
                         
                         lower_beta, 
                         
                         upper_beta, 
                         
                         lower_t_tr, 
                         
                         upper_t_tr ):
    
    """Repeat the fitting process until the variation is reduced to N_SF significant figures. N_Repeats is the number of 
    
    repeats that should be made about the sample"""
        
    Treated_Pairs = {}
    
    Tested_betas = []
    Tested_t_trs = []
    Tested_RSSs = []    
    
    Variation = ( upper_beta - lower_beta ) / upper_beta
    
    for l in range( N_repeats ):
        
        # Run Fitting Process 
        
        Additional_Pairs = beta_measured_fitter( n_CEs, 
                                         
                                         G_lights, 
                                         
                                         measured_betas, 
                                         
                                         N_samples, 
                                         
                                         lower_beta, 
                                         
                                         upper_beta, 
                                         
                                         lower_t_tr, 
                                         
                                         upper_t_tr )
        
        # Extract Parameters
    
        Additional_betas = []
        Additional_t_trs = []
        Additional_RSSs = []

        for Pair in Additional_Pairs.keys():
    
            Entry = Additional_Pairs[ Pair ]
    
            Additional_betas.append( Entry[ 0 ] )
    
            Additional_t_trs.append( Entry[ 1 ] )

            Additional_RSSs.append( Entry[ 2 ] )
            
        Tested_betas = Tested_betas + Additional_betas
        
        Tested_t_trs = Tested_t_trs + Additional_t_trs
   
        Tested_RSSs = Tested_RSSs + Additional_RSSs
            
        # Determine Which Parameter Values Give Min RSS
    
        Index_of_Min_RSS = where( Additional_RSSs == min( Additional_RSSs ) )[ 0 ][ 0 ]
        
        New_beta = Additional_betas[ Index_of_Min_RSS ]
        
        New_t_tr = Additional_t_trs[ Index_of_Min_RSS ]   
        
        # Set New Limits:
        
        lower_beta = New_beta * 0.1
        
        upper_beta = New_beta * 1.5
                                         
        lower_t_tr = New_t_tr * 0.5
        
        upper_t_tr = New_t_tr * 1.5

    Minimal_RSS_index = where( Tested_RSSs == min( Tested_RSSs ) )[ 0 ][ 0 ]
    
    Output_beta = Tested_betas[ Minimal_RSS_index ]
    
    Output_t_tr = Tested_t_trs[ Minimal_RSS_index ]

    return Output_beta, Output_t_tr, Tested_betas, Tested_t_trs, Tested_RSSs

<a id = 'BetaUIComp' ></a>
### 3.2. User Interface Compiling
[Return to Table of Contents](#ToC)
<br/><br/> 
To begin, widgets for loading and viewing experimental data are specified, starting with a widget for specifying the filename:
<br/><br/> 

In [14]:
Filename_Selector = Combobox( 
    
    value = None,
    
    placeholder = 'Specify Filename',

    options = Beta_Files )

<br/><br/> 
Following this, an error label is defined for telling the User that they haven't specified a correct filename: 
<br/><br/> 

In [15]:
Error_Label = Valid( 
    
    value = False, 
    
    readout = 'Invalid filename',

    layout = Layout(
    
        width = '100%' ) )

Error_Label.layout.display = 'none'  # Initially hide the error label

<br/><br/> 
This error label will be revealed when the User presses the following button without specifying a valid filename:
<br/><br/> 

In [16]:
Add_File_Button = Button( description = 'Add Data' )

<br/><br/> 
This button is instructed to obey the following function:
<br/><br/> 

In [17]:
def Valid_Filename( Button ):
    
    """If a valid filename has been entered, load the data."""
    
    Data_Name = Filename_Selector.value
    
    if Data_Name in Beta_Files:
        
        # Hide error label if it's revealed
        
        Error_Label.layout.display = 'none'
        
        # Remove data from box
        
        Filename_Selector.value = ''
        
        # Add data to tab of available data
        
        Data_Loader( Data_Name )
        
    else:
        
        Error_Label.layout.display = None
        
Add_File_Button.on_click( Valid_Filename )

<br/><br/> 
After clicking this button, the function "Data_Loader" defined shortly is called. This will load in the valid data, store it in a dictionary, and generate a UI for fitting it. These UIs are stored in the following "Tab" widget:
<br/><br/> 

In [18]:
Data_Fitting_Tab = Tab()

Data_Fitting_Tab.layout.display = 'none'

<br/><br/> 
The data-loading function is then defined as:
<br/><br/> 

In [19]:
Loaded_Data = {}

Fitting_Data = {}

def Data_Loader( Data_Name ):
    
    """Load the data using the User's specified filename, call the functions that allow the User to (i) customise the data,
    
    (2) view the UI for fitting, and (iii) copy the results."""
        
    global Loaded_Data
    
    Data = read_excel( Beta_Filepaths[ Data_Name ] )
    
    # Assumes columns are organised as: n_CE, G_light, Beta 
    
    Column_Titles = list( Data.columns )
    
    n_CEs = list( Data[ Column_Titles[ 0 ] ] )
    
    G_lights = list( Data[ Column_Titles[ 1 ] ] )

    Betas = list( Data[ Column_Titles[ 2 ] ] )
    
    # Store Data in Loaded_Data dictionary, Make Sure the Name is Unique
        
    Stored_Data_Names = Loaded_Data.keys()
    
    if Data_Name in Stored_Data_Names:
        
        highest = 0

        for j in range( len( Stored_Data_Names ) ):
    
            if Data_Name + '_' + str( j ) in Stored_Data_Names:
        
                highest += 1
        
        New_Data_Name = Data_Name + '_' + str( highest )    
    
    else:
        
        New_Data_Name = Data_Name 
        
    # Store the data in the "Loaded_Data" dictionary
    
    Loaded_Data[ New_Data_Name ] = { 'n_CE' : n_CEs, 'G_light' : G_lights, 'Beta' : Betas }
    
    # Also store it in the "Fitting_Data" dictionary (code assumed that all points should be fitted)
    
    Fitting_Data[ New_Data_Name ] = { 'n_CE' : n_CEs, 'G_light' : G_lights, 'Beta' : Betas }
    
    # Generate Table, Plot, and Fitting UI
    
    Data_Fitting_Tab.layout.display = None  # Reveal tab if it hasn't been revealed already
    
    # Create Table
    
    Table = Table_Maker( New_Data_Name )
    
    # Create Plot and Store it
    
    Plot_Maker( New_Data_Name )
    
    # Create the Fitting UI
    
    Fitting_UI_Maker( New_Data_Name, Table )
    
    Compiled_UI = Fitting_UIs[ New_Data_Name ]
    
    # Append this Tab to Data_Fitting_Tab.
    
    Number_of_Tabs = len( Data_Fitting_Tab.children )
        
    Data_Fitting_Tab.children = Data_Fitting_Tab.children + ( Compiled_UI, )
    
    Data_Fitting_Tab.set_title( Number_of_Tabs, New_Data_Name )
    
    Data_Fitting_Tab.selected_index = Number_of_Tabs

<br/><br/> 
Three additional functions are defined in this function - These are defined in the remainder of this section. One generates the table (such that data can be selected and anomalies removed), one generates the UI for fitting, and one generates a plot. These functions use further, supplementary functions; these are defined as and when needed.
<br/><br/> 

In [20]:
Data_Checkboxes = {}

def Table_Maker( Data_Name ):
    
    """Load the data using the User's specified filename, generate a table that allows them to customise the data, remove
    
    anomalies, etc."""
    
    Data = Loaded_Data[ Data_Name ]
    
    n_CEs = Data[ 'n_CE' ]
    
    G_lights = Data[ 'G_light' ]
    
    Betas = Data[ 'Beta' ]
    
    # First, Determine the Number of Data Points and Generate a Checkbox for Each of the Data Points
    
    global Data_Checkboxes
    
    N_Points = len( n_CEs )
    
    Checkboxes = [ Checkbox( 
    
            value = True,
        
            layout = Layout( 
            
                width = '25%',
                
                #display = 'flex',
            
                justify_content = 'flex-start' ),        
        
            description = '',
    
            style = { '_view_name' : str( j ) } ) for j in range( N_Points ) ]  # Checkboxes are stored according to int
        
    Data_Checkboxes[ Data_Name ] = Checkboxes
    
    # These checkboxes are instructed to obey the function "Refresh_UI", such that, when they are changed, the data saved
    
    # in the "Fitting_Data" dictionary is also changed
    
    for j in range( N_Points ): 
        
        Checkboxes[ j ].observe( Refresh_UI, names = 'value' )
    
    # Next, generate labels illustrating each of the data point:
    
    n_CE_Labels =  [ Label( 
    
            value = '%.5E' % Decimal( n_CEs[ j ] ),
        
            layout = Layout( 
            
                width = '25%',
                
                display = 'flex',
            
                justify_content = 'center' ),
            
            style = { '_view_name' : str( j ) } ) for j in range( N_Points ) ]  # Labels are stored according to int
        
    G_light_Labels =  [ Label( 
    
            value = '%.5E' % Decimal( G_lights[ j ] ),
       
            layout = Layout( 
            
                width = '25%',
                
                display = 'flex',
            
                justify_content = 'center' ),        
            
            style = { '_view_name' : str( j ) } ) for j in range( N_Points ) ]  # Labels are stored according to int
                
    Beta_Labels =  [ Label( 
    
            value = '%.5E' % Decimal( Betas[ j ] ),
        
            layout = Layout(
                
                width = '25%',
                
                display = 'flex',
            
                justify_content = 'center' ),        
            
            style = { '_view_name' : str( j ) } ) for j in range( N_Points ) ]  # Labels are stored according to int
    
    Grid = GridspecLayout( N_Points + 1 , 4 )
    
    Column_Titles = HBox( [ Label( 'Include',
                                 
                                layout = Layout(
                
                                    width = '25%',
                
                                    display = 'flex',
            
                                    justify_content = 'center' ),  ), 
                           
                           Label( '$n_{\mathrm{CE}}$',
                                 
                                layout = Layout(
                
                                    width = '25%',
                
                                    display = 'flex',
            
                                    justify_content = 'center' ),  ), 
                           
                           Label( '$G_{\mathrm{light}}$',
                                 
                                layout = Layout(
                
                                    width = '25%',
                
                                    display = 'flex',
            
                                    justify_content = 'center' ),  ), 
                           
                           Label('$\\beta$',
                                 
                                layout = Layout(
                
                                    width = '25%',
                
                                    display = 'flex',
            
                                    justify_content = 'center' ),  ), ] )

    Grid[ 0 , : ] = Column_Titles
    
    for i in range( N_Points ):
        
#        Column = Columns[ i ]
        
        Grid[ i + 1 , : ] = HBox( [
            
            Checkboxes[ i ], 
            
            n_CE_Labels[ i ],
        
            G_light_Labels[ i ],
            
            Beta_Labels[ i ] ] )
        
    return Grid

<br/><br/> 
Next comes the widgets for fitting the data and plotting the graph. To start, input boxes are defined for specifying the valeus of the transit time and bimolecular recombination rate constant, as well as their limits for the fitting. The graph defaults to using these initial input values. A function that may be called to generate these widgets (for a given data set) is defined as:
<br/><br/> 

In [21]:
Fit_Parameter_Input_Widgets = {}

Number_of_Samples = BoundedIntText(

    value = 10000,

    min = 1,

    max = 1e100,

    description = 'Number of Samples (Log-Scale)',

    style = { 'description_width' : 'initial' } )

Number_of_Repeats = BoundedIntText(

    value = 1,

    min = 1,

    max = 1e100,

    description = 'Number of Re-samples',

    style = { 'description_width' : 'initial' } ) 

Number_of_Samples.layout.display = 'none'
Number_of_Repeats.layout.display = 'none'

def Fit_Parameter_Input_Widget_Generator( Data_Name ):
    
    """Upon being called, generate wigets for customising the values of the fit parameters, and store these in a dictionary.
    
    """

    Fit_Beta = BoundedFloatText(

        value = 1.2e-11,  # Input value

        min = 0,

        max = 1e100,

        description = 'BR rate constant (cm3/s)',

        style = { 'description_width' : 'initial' } )

    Fit_t_tr = BoundedFloatText(

        value = 1e-7,

        min = 0,

        max = 1e100,

        description = 'Transit time (s)',

        style = { 'description_width' : 'initial' } )
    
    Min_Fit_Beta = BoundedFloatText(

        value = 1e-12,  # Input value

        min = 0,

        max = 1e100,

        description = 'Min. BR rate constant (cm3/s)',

        style = { 'description_width' : 'initial' } )

    Max_Fit_Beta = BoundedFloatText(

        value = 1e-10,  # Input value

        min = 0,

        max = 1e100,

        description = 'Max. BR rate constant (cm3/s)',

        style = { 'description_width' : 'initial' } )    
    
    Min_t_tr = BoundedFloatText(

        value = 1e-9,

        min = 0,

        max = 1e100,

        description = 'Min. transit time (s)',

        style = { 'description_width' : 'initial' } )      
    
    Max_t_tr = BoundedFloatText(

        value = 1e-5,

        min = 0,

        max = 1e100,

        description = 'Max. transit time (s)',

        style = { 'description_width' : 'initial' } )   
    
    # The fit widgets are initially hidden until the following button is pressed to show them
    
    Min_Fit_Beta.layout.display = 'none'
    
    Max_Fit_Beta.layout.display = 'none'

    Min_t_tr.layout.display = 'none'
    
    Max_t_tr.layout.display = 'none'
    
    Show_Limits_Checkbox = Checkbox(
        
        value = False,
    
        description = 'Show Fitting Limits',

        style = { 'description_width' : 'initial',
                 
                 '_view_name' : Data_Name } )    

    Show_Limits_Checkbox.observe( Show_Parameter_Limits, names = 'value' )
    
    global Fit_Parameter_Input_Widgets
    
    Fit_Parameter_Input_Widgets[ Data_Name ] = {
        
        'beta' : Fit_Beta,
        
        'min beta' : Min_Fit_Beta,
        
        'max beta' : Max_Fit_Beta,
        
        't_tr' : Fit_t_tr,
        
        'min t_tr' : Min_t_tr,

        'max t_tr' : Max_t_tr,
        
    }
    
    return VBox( [
        
        Fit_Beta,
        
        Fit_t_tr,
        
        Show_Limits_Checkbox,
        
        Min_Fit_Beta,
        
        Max_Fit_Beta,
        
        Min_t_tr,
        
        Max_t_tr,
        
        Number_of_Samples,
        
        Number_of_Repeats,
        
    ])

<br/><br/> 
The following function is defined for showing the fitting limits (which are hidden by default):
<br/><br/> 

In [22]:
def Show_Parameter_Limits( Change ):
    
    """Show the widgets for customising the fit parameter limits on change, or hide them."""
    
    Data_Name = Change.owner.style._view_name 
    
    Fit_Widgets = Fit_Parameter_Input_Widgets[ Data_Name ]
    
    if Change[ 'new' ] == True:
        
        Fit_Widgets[ 'min beta' ].layout.display = None
        
        Fit_Widgets[ 'max beta' ].layout.display = None

        Fit_Widgets[ 'min t_tr' ].layout.display = None

        Fit_Widgets[ 'max t_tr' ].layout.display = None
        
        Number_of_Samples.layout.display = None
        
        Number_of_Repeats.layout.display = None
        
    else:
        
        Fit_Widgets[ 'min beta' ].layout.display = 'none'
        
        Fit_Widgets[ 'max beta' ].layout.display = 'none'

        Fit_Widgets[ 'min t_tr' ].layout.display = 'none'

        Fit_Widgets[ 'max t_tr' ].layout.display = 'none'   
        
        Number_of_Samples.layout.display = 'none'
        
        Number_of_Repeats.layout.display = 'none'

<br/><br/> 
A few functions are now used to change the UI at the User's request, the first of these refreshes the UI if the User chooses to remove an anomalous data point, or reconduct a fitting:
<br/><br/> 

In [23]:
def Refresh_UI( Change ):
    
    """Upon being called, refresh the UI for correcting beta vs n_CE data using the data stored in the Fitting_Data 
    
    dictionary."""
    
    # Determine selected file from open tab:
    
    Data_Name = Data_Fitting_Tab._titles[ str( Data_Fitting_Tab.selected_index ) ]
    
    # Load Data:
    
    Data = Loaded_Data[ Data_Name ]
    
    n_CEs = Data[ 'n_CE' ]
    
    G_lights = Data[ 'G_light' ]
    
    Betas = Data[ 'Beta' ]
    
    N_Points = len( n_CEs )
    
    # Determine which widgets are selected:
    
    Valid_Indices = [ j for j in range( N_Points ) if Data_Checkboxes[ Data_Name ][ j ].value == True ]
    
    # Store Fitting Data Again
    
    Fit_n_CEs = [ n_CEs[ j ] for j in Valid_Indices ]
    
    Fit_G_lights = [ G_lights[ j ] for j in Valid_Indices ]

    Fit_Betas = [ Betas[ j ] for j in Valid_Indices ]
    
    Fitting_Data[ Data_Name ] = { 
    
        'n_CE' : Fit_n_CEs,
    
        'G_light' : Fit_G_lights,
    
        'Beta' : Fit_Betas }
    
    # Regenerate Plot
    
    Plot_Maker( Data_Name )
    
    # Update UI
    
    Fitting_UIs[ Data_Name ].children = ( Graphs[ Data_Name ], ) + Fitting_UIs[ Data_Name ].children[ 1:]

<br/><br/> 
Upon pressing either of the fitting buttons, the following function is called to fit the data (then refresh the UI using the above function): 
<br/><br/> 

In [24]:
Fit_Parameters_Dictionary = {}

Optimised_Fitting_Values = {}

def Data_Fitter( Button ):
    
    """Fit the data when called and update the parameter values."""
    
    # Load Data
    
    Data_Name = Button.style._view_name
    
    Data = Fitting_Data[ Data_Name ]
        
    n_CEs = array( Data[ 'n_CE' ] )
    
    Betas = array( Data[ 'Beta' ] )
    
    G_lights = array( Data[ 'G_light' ] )
    
    # Load Parameters
    
    global Fit_Parameter_Input_Widgets
    
    Fit_Parameters = Fit_Parameter_Input_Widgets[ Data_Name ]
    
    # Fit Data Using Stored Parameters
    
    if Button.description == 'Fit Data':
        
        Beta = Fit_Parameters[ 'beta' ].value
        
        t_tr = Fit_Parameters[ 't_tr' ].value
            
    else:        
        
        Fit_Parameters = Fit_Parameter_Input_Widgets[ Data_Name ]
    
        Beta, t_tr, Tested_betas, Tested_t_trs, Tested_RSSs = Repeated_Beta_fitter( n_CEs, 
                         
                         G_lights, 
                         
                         Betas, 
                         
                         Number_of_Samples.value, 
                         
                         Number_of_Repeats.value,
                         
                         Fit_Parameters[ 'min beta' ].value, 
                         
                         Fit_Parameters[ 'max beta' ].value, 
                         
                         Fit_Parameters[ 'min t_tr' ].value, 
                         
                         Fit_Parameters[ 'max t_tr' ].value )
                
        Fit_Parameter_Input_Widgets[ Data_Name ][ 'beta' ].value = Beta
        
        Fit_Parameter_Input_Widgets[ Data_Name ][ 't_tr' ].value = t_tr
        
        UI_Buttons[ Data_Name ][ 'Optimise' ].button_style = 'success'
        
        global Optimised_Fitting_Values

        Optimised_Fitting_Values[ Data_Name ] = { 'beta' : Tested_betas, 't_tr' : Tested_t_trs, 'RSS' : Tested_RSSs }
        
        UI_Buttons[ Data_Name ][ 'Copy RSS' ].disabled = False

    global Fit_Parameters_Dictionary
    
    UI_Buttons[ Data_Name ][ 'Copy Data' ].disabled = False
   
    UI_Buttons[ Data_Name ][ 'Copy Parameters' ].disabled = False

    Fit_Parameters_Dictionary[ Data_Name ] = { 'beta' : Beta, 't_tr' : t_tr }
    
    # Refresh the UI (Plot_Maker will Run the Fitting)
    
    Refresh_UI( '' )

<br/><br/> 
The following function creates the graph:
<br/><br/> 

In [25]:
Graphs = {}

Correction_Factors = {}

Correction_Betas = {}

def Plot_Maker( Data_Name ):
    
    """Plot a graph of the uncorrected data, and the corrected data and beta value."""
    
    Data = Fitting_Data[ Data_Name ]
    
    n_CEs = array( Data[ 'n_CE' ] )
    
    Betas = array( Data[ 'Beta' ] )
    
    G_lights = array( Data[ 'G_light' ] )
    
    Graph = Output()
    
    with Graph:
        
        plt.plot( n_CEs, Betas, '.', label = 'Loaded $\\beta$' )
        
        plt.xscale( 'log' )
        
        plt.yscale( 'log' )
    
        plt.ylabel( '$\\beta$ (cm$^{3}$ s$^{-1}$)') 
        
        plt.xlabel( '$n_{\mathrm{CE}}$ (cm${-3}$)')
        
        plt.grid( which = 'both' )
            
        if Data_Name in Fit_Parameters_Dictionary.keys():
        
            # Plot the corrected data if values available
        
            Fit_Beta = Fit_Parameters_Dictionary[ Data_Name ][ 'beta' ]
    
            Fit_t_tr = Fit_Parameters_Dictionary[ Data_Name ][ 't_tr' ]
    
            # Correct Data
    
            Corrections = beta_correction( n_CEs, Fit_t_tr, G_lights, Fit_Beta )
        
            global Correction_Factors, Correction_Betas
            
            Correction_Factors[ Data_Name ] = Corrections
        
            Corrected_Betas = Betas * Corrections
            
            Correction_Betas[ Data_Name ] = Corrected_Betas
        
            plt.plot( n_CEs, [ Fit_Beta for n_CE in n_CEs ], '--', label = 'Fit $\\beta$')
        
            plt.plot( n_CEs, Corrected_Betas, '.', label = 'Corrected $\\beta$' )
            
            plt.plot( n_CEs, beta_measured( n_CEs, G_lights, Fit_Beta, Fit_t_tr ), label = 'Re-Simulated $\\beta$' )
                    
            plt.legend()
            
        plt.show()

    global Graphs
    
    Graphs[ Data_Name ] = Graph 

<br/><br/> 
Next comes three functions for copying data. One copies the parameters, one copies raw RSS data from the fitting, and the last copies the corrected recombination rate constant data.
<br/><br/> 

In [26]:
def Parameter_Copier( Button ):
    
    """Copy the parameters optimised from the fitting"""
    
    Data_Name = Button.style._view_name
    
    Params = Fit_Parameters_Dictionary[ Data_Name ]
    
    DataFrame( { 
        
        'Parameter' : [ 'beta', 't_tr' ], 
        
        'Value' : [ Params[ 'beta' ], Params[ 't_tr' ] ] } ).to_clipboard( index = False )
    
def Data_Copier( Button ):
    
    """Copy the data optimised from the fitting."""
    
    Data_Name = Button.style._view_name
    
    Fit_Params = Fitting_Data[ Data_Name ]
    
    Fit_Params = Fit_Params | { 'Correction factor' : Correction_Factors[ Data_Name ],
                               
                               'Corrected betas' : Correction_Betas[ Data_Name ] }
    
    DataFrame( Fit_Params ).to_clipboard( index = False )
    
def RSS_Data_Copier( Button ):
    
    """Copy the data optimised from the fitting."""
    
    Data_Name = Button.style._view_name
    
    DataFrame( Optimised_Fitting_Values[ Data_Name ] ).to_clipboard( index = False )

<br/><br/> 
Finally, the following function ties everything together to create the UI:
<br/><br/> 

In [27]:
Fitting_UIs = {}

UI_Buttons = {}

def Fitting_UI_Maker( Data_Name, Table ):
    
    """Generate the UI for fitting the data."""
    
    Fitting_Widgets = Fit_Parameter_Input_Widget_Generator( Data_Name )
    
    # Data Fitting Buttons
    
    Run_Fitting_Button = Button(

        description = 'Fit Data',
    
        style = { '_view_name' : Data_Name } )
    
    Run_Fitting_Button.on_click( Data_Fitter )
    
    Optimise_Parameter_Button = Button( 

        description = 'Optimise Parameters',

        button_style = 'warning',
        
        style = { '_view_name' : Data_Name } )  
    
    Optimise_Parameter_Button.on_click( Data_Fitter )
        
    # Data Copying Butotns

    Copy_Parameters_Button = Button(

        description = 'Copy Fit Parameters',
        
        style = { '_view_name' : Data_Name },
    
        disabled = True )

    Copy_Parameters_Button.on_click( Parameter_Copier )    
    
    Copy_RSS_Button = Button(

        description = 'Copy RSS Data',
        
        style = { '_view_name' : Data_Name },
    
        disabled = True )
    
    Copy_RSS_Button.on_click( RSS_Data_Copier )
    
    Copy_Data_Button = Button(

        description = 'Copy Data',
        
        style = { '_view_name' : Data_Name },
    
        disabled = True )   
    
    Copy_Data_Button.on_click( Data_Copier )
    
    # These widgets are stored so they can be revealed and altered when required
    
    UI_Buttons[ Data_Name ] = { 'Optimise' : Optimise_Parameter_Button, 
                               
                               'Copy Data' : Copy_Data_Button,
                               
                               'Copy Parameters' : Copy_Parameters_Button,
                              
                                'Copy RSS' : Copy_RSS_Button }
    
    # Compile all control widgets into a box
    
    Control_Widgets =  VBox( [ Fitting_Widgets,
       
       HBox( [ Run_Fitting_Button, Optimise_Parameter_Button ] ),
              
       HBox( [ Copy_Data_Button, Copy_Parameters_Button, Copy_RSS_Button ] ) ] )
        
    # Store the box in a tab with the table containing the data
    
    Fitting_Tab = Tab()
    
    Fitting_Tab.children = ( Table, Control_Widgets )
    
    Fitting_Tab.set_title( 0, 'Data' )
    
    Fitting_Tab.set_title( 1, 'Fitting' )   
    
    # Compile and Store UI
    
    Compiled_UI = VBox( [
        
        Graphs[ Data_Name ],
        
        Fitting_Tab ] )
    
    global Fitting_UIs
    
    Fitting_UIs[ Data_Name ] = Compiled_UI

<a id = 'BetaUI' ></a>
### 3.3. The User Interface
[Return to Table of Contents](#ToC)

In [28]:
Data_Loading_Box = VBox( [ 
    
    Filename_Selector,

    Error_Label,
    
    Add_File_Button,
    
    Data_Fitting_Tab

] )

Data_Loading_Box